# 第7章 推荐系统实例

## 外围架构

<br/><center>
<img style="border-radius: 0.3125em;box-shadow: 0 2px 4px 0 rgba(34,36,38,.12),0 2px 10px 0 rgba(34,36,38,.08);" src="images/7-1-recommend-system-relationship.png"><br><div style="color:orange; border-bottom: 1px solid #d9d9d9;display: inline-block;color: #000;padding: 2px;">图7.1 推荐系统和其他系统之间的关系</div></center>

&emsp;&emsp;每个网站都会有一个UI系统， UI系统负责给用户展示网页并和用户交互。网站会通过日志系统将用户在UI上的各种各样的行为记录到用户行为日志中。日志可能存储在内存缓存里，也可能存储在数据库中，也可能存储在文件系统中。而推荐系统通过分析用户的行为日志，给用户生成推荐列表，最终展示到网站的界面上。  
&emsp;&emsp;目前流行的推荐系统界面，有以下的共性：
- 通过一定方式展示物品，主要包括物品的标题、缩略图和介绍等
- 很多推荐界面都提供了推荐理由，理由可以增加用户对推荐结果的信任度
- 推荐界面还需要提供一些按钮让用户对推荐结果进行反馈，这样才能让推荐算法不断改善用户的个性化推荐体验

&emsp;&emsp;按照数据的规模和是否需要实时存取，不同的行为数据将被存储在不同的媒介中。一般来说，需要实时存取的数据存储在数据库和缓存中，而大规模的非实时地存取数据存储在分布式文件系统（如HDFS）中。

## 推荐系统架构

推荐系统联系用户和物品的方式主要有3种：

<br/><center>
<img style="border-radius: 0.3125em;box-shadow: 0 2px 4px 0 rgba(34,36,38,.12),0 2px 10px 0 rgba(34,36,38,.08);" src="images/7-2-three-connections.png"><br><div style="color:orange; border-bottom: 1px solid #d9d9d9;display: inline-block;color: #000;padding: 2px;">图7.2 3种联系用户和物品的推荐系统</div></center>

主要的用户的特征种类：
- 人口统计学特征：包括用户的年龄、性别、国籍和民族等用户在注册时提供的信息
- 用户的行为特征：包括用户浏览过什么物品、收藏过什么物品、给什么物品打过什么样的分数等用户行为相关的特征。同时，用户行为从时间上也可以分为用户近期的行为和长期的行为
- 用户的话题特征：可以根据用户的历史行为利用话题模型（topic model）将电视剧和电影聚合成不同的话题，并且计算出每个用户对什么话题感兴趣

推荐系统的推荐任务：
- 将最新加入的物品推荐给用户
- 将商业上需要宣传的物品推荐给用户
- 给用户推荐不同种类的物品
- 给用户混合推荐
- 对于不同的产品推荐不同新颖度的物品
- 考虑到用户访问推荐系统的上下文

&emsp;&emsp;推荐系统需要由多个推荐引擎组成，每个推荐引擎负责一类特征和一种任务，而推荐系统的任务只是将推荐引擎的结果按照一定权重或者优先级合并、排序然后返回。

<br/><center>
<img style="border-radius: 0.3125em;box-shadow: 0 2px 4px 0 rgba(34,36,38,.12),0 2px 10px 0 rgba(34,36,38,.08);" src="images/7-3-recommended-system-architecture.png"><br><div style="color:orange; border-bottom: 1px solid #d9d9d9;display: inline-block;color: #000;padding: 2px;">图7.3 推荐系统的架构图</div></center>

优点：
- 可以方便地增加/删除引擎，控制不同引擎对推荐结果的影响。对于绝大多数需求，只需要通过不同的引擎组合实现。
- 可以实现推荐引擎级别的用户反馈。每一个推荐引擎其实代表了一种推荐策略，而不同的用户可能喜欢不同的推荐策略。

## 推荐引擎的架构

<br/><center>
<img style="border-radius: 0.3125em;box-shadow: 0 2px 4px 0 rgba(34,36,38,.12),0 2px 10px 0 rgba(34,36,38,.08);" src="images/7-4-recommended-engine-architecture.png"><br><div style="color:orange; border-bottom: 1px solid #d9d9d9;display: inline-block;color: #000;padding: 2px;">图7.4 推荐引擎的架构图</div></center>

- A部分负责从数据库或者缓存中拿到用户行为数据，通过分析不同行为，生成当前用户的特征向量。不过如果是使用非行为特征，就不需要使用行为提取和分析模块了。该模块的输出是用户特征向量。
- B部分负责将用户的特征向量通过特征-物品相关矩阵转化为初始推荐物品列表。
- C部分负责对初始的推荐列表进行过滤、排名等处理，从而生成最终的推荐结果。

### 生成用户特征向量

特征向量由特征以及特征的权重组成，在利用用户行为计算特征向量时，需要考虑以下因素：
- 用户行为的种类
- 用户行为产生的时间
- 用户行为的次数
- 物品的热门程度

推荐引擎在生成用户特征时，会加重不热门物品对应的特征权重。

### 特征-物品相关推荐

&emsp;&emsp;对于每个特征，可以在相关表中存储和它最相关的N个物品的ID。  
&emsp;&emsp;特征—物品相关推荐模块还可以接受一个候选物品集合。候选物品集合的目的是保证推荐结果只包含候选物品集合中的物品。  

### 过滤模块

在得到初步的推荐列表后，还不能把这个列表展现给用户，首先需要按照产品需求对结果进行过滤，过滤掉那些不符合要求的物品。  
主要过滤以下物品：
- 用户已经产生过行为的物品
- 候选物品以外的物品
- 某些质量很差的物品

### 排名模块

#### 新颖性排名
&emsp;&emsp;新颖性排名模块的目的是给用户尽量推荐他们不知道的、长尾中的物品。要准确了解用户是否已经知道某个物品是非常困难的，因此我们只能通过某种近似的方式知道，比如对推荐结果中热门的物品进行降权：$$p_{ui}=\frac{p_{ui}}{\log (1 + \alpha \cdot \text{popularity}(i))}$$  
&emsp;&emsp;本章提到的推荐系统架构主要是基于物品的推荐算法，基于物品的推荐算法基本公式如下：$$p_{ui}=\sum_{j \in N(u) \cap S(i,K)} w_{ji} r_{uj}$$其中，$j$就是联系用户和推荐物品的特征。  
&emsp;&emsp;如果要提高推荐结果的新颖度，就需要对$r_{uj}$进行降权：$$r_{uj} = \frac{r_{uj}}{\log (1 + \alpha \cdot \text{popularity}(j))}$$  
&emsp;&emsp;推荐系统是为了给用户介绍他们不熟悉的物品，那么可以假设如果用户知道了物品$j$，对物品$j$产生过行为，那么和$j$相似的且比$j$热门的物品用户应该也有比较大的概率知道，因此可以降低这种物品的权重：$$w_{ji} = \frac{w_{ji}}{\log (1 + \alpha \cdot \text{popularity}(i))} \quad (\text{popularity}(i) > \text{popularity}(j))$$  

#### 多样性

&emsp;&emsp;提高多样性的方法：
- 将推荐结果按照某种物品的内容属性分成几类，然后在每个类中都选择该类中排名最高的物品组合成最终的推荐列表。  
- 控制不同推荐结果的推荐理由出现的次数。

#### 时间多样性

&emsp;&emsp;时间多样性主要是为了保证用户不要每天来推荐系统都看到同样的推荐结果。  
&emsp;&emsp;提高时间多样性的方法：
1. 保证推荐系统的实时性
2. 在用户没有新的行为时，也要保证推荐结果每天都有变化：
 - 记录用户每次登陆推荐系统看到的推荐结果。
 - 将这些结果发回日志系统。这种数据不需要实时存储，只要能保证小于一天的延时就足够了。
 - 在用户登录时拿到用户昨天及之前看过的推荐结果列表，从当前推荐结果中将用户已经看到的推荐结果降权。

#### 用户反馈
&emsp;&emsp;用户反馈模块主要通过分析用户之前和推荐结果的交互日志，预测用户会对什么样的推荐结果比较感兴趣。  
&emsp;&emsp;如果推荐系统的目标是提高用户对推荐结果的点击率，那么可以利用点击模型（click model）预测用户是否会点击推荐结果。  
&emsp;&emsp;点击预测的主要问题是预测用户看到某个推荐结果时是否会点击。那么要进行点击率预测，首先需要提取特征。点击模型需要离线计算好，在线将模型加载到内存中。为了提高在线预测的效率，一般只可以使用线性模型。